In [63]:
#Change the path to where your model is
PATH_TO_MODEL = '/punctRestorationModel.pth'

##IMPORTS

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 7.6 MB/s 
     |████████████████████████████████| 596 kB 67.4 MB/s 
     |████████████████████████████████| 101 kB 13.0 MB/s 
     |████████████████████████████████| 6.6 MB 44.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
# Importing the relevant modules
from transformers import BertTokenizer, BertModel
import pandas as pd
import numpy as np
import torch
import re

In [3]:
from sklearn.metrics import classification_report, f1_score
from sklearn.utils import class_weight

import numpy as np
import copy
import random
import time

import torch
from torch import nn
from torch.nn import functional as F
import torch.optim as optim
from torch.utils.data import (TensorDataset, DataLoader, RandomSampler, SequentialSampler)

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

##Download model

In [40]:
class CNN(nn.Module):
    def __init__(self, embedding_dim, n_filters, filter_sizes, output_dim, dropout, sent_len):
        super().__init__()
        self.sent_len = sent_len
        
        self.convs = nn.ModuleList([
                                    nn.Conv2d(in_channels = 1, 
                                              out_channels = n_filters, 
                                              kernel_size = (sent_len//2, embedding_dim//fs),
                                              stride = embedding_dim//fs//2
                                              ) 
                                    for fs in filter_sizes
                                    ])
        
        self.convs2 = nn.ModuleList([
                                    nn.Conv2d(in_channels = 1, 
                                              out_channels = n_filters, 
                                              kernel_size = (sent_len//2, embedding_dim//fs),
                                              stride = embedding_dim//fs//2
                                              ) 
                                    for fs in filter_sizes
                                    ])
        
        self.convs3 = nn.ModuleList([
                                    nn.Conv2d(in_channels = 1, 
                                              out_channels = n_filters, 
                                              kernel_size = (2, embedding_dim//fs),
                                              stride = embedding_dim//fs//2
                                              ) 
                                    for fs in filter_sizes
                                    ])
        
        
        self.fc = nn.Linear(len(filter_sizes) * n_filters*3, len(filter_sizes) * n_filters)
        self.fc2 = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        self.out = nn.Softmax(dim=0)
        
    def forward(self, text):
        embedded = text        
        embedded = embedded.unsqueeze(1)
        
        conved = [F.elu(conv(embedded[:,:,6//2:,:])).squeeze(2) for conv in self.convs]
        conved2 = [F.elu(conv(embedded[:,:,:6//2,:])).squeeze(2) for conv in self.convs2]
        conved3 = [F.elu(conv(embedded[:,:,6//2-1:6//2+1,:])).squeeze(2) for conv in self.convs3]
                
        pooled = [F.avg_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        pooled2 = [F.avg_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved2]
        pooled3 = [F.avg_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved3]
        
        cat = self.dropout(torch.cat(pooled+pooled2+pooled3, dim = 1))

        logits = self.dropout(self.fc(cat))
        logits = self.fc2(logits)

            
        return logits

In [42]:
EMBEDDING_DIM = 768
N_FILTERS = 25
FILTER_SIZES = [1,2,4,6,8,12,16]
DROPOUT = 0.5
OUTPUT_DIM = 3

modelPunct = CNN(EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT, 6)
modelPunct.load_state_dict(torch.load(PATH_TO_MODEL,map_location=torch.device('cpu')))

<All keys matched successfully>

In [41]:
modelBert = BertModel.from_pretrained("DeepPavlov/rubert-base-cased", output_hidden_states = True,)

tokenizer = BertTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [43]:
def bert_text_preparation(text, tokenizer):
    marked_text = "[CLS] " + text + " [SEP]"
    tokenized_text = tokenizer.tokenize(marked_text)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    segments_ids = [1]*len(indexed_tokens)


    # word_lens stores length of each word in input text, because some words get broken into multiple tokens

    word_lens = []
    for i in range(len(tokenized_text)):
      token = tokenized_text[i]
      if token[0:2] == "##":
        word_lens[-1]+=1
      elif tokenized_text[i-1] == '-':
        word_lens[-2] += 2
        word_lens.pop()
      else:
        word_lens.append(1)

    # Convert inputs to PyTorch tensors
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])

    return tokenized_text, tokens_tensor, segments_tensors, word_lens
    
def get_bert_embeddings(tokens_tensor, segments_tensors, word_lens, model):
    with torch.no_grad():
        outputs = model(tokens_tensor, segments_tensors)
        # Removing the first hidden state
        # The first state is the input state
        hidden_states = outputs[2][1:]

    token_embeddings = torch.stack(hidden_states, dim=0)
    token_embeddings = torch.squeeze(token_embeddings, dim=1)
    token_embeddings = token_embeddings.permute(1,0,2)


    # Get embeddings from the last 3 layers of BERT and concatenate them 
    # For each words that is longer than one token, take the mean of all the token
    # The resulting embedding size is 2304 for each word

    list_token_embeddings = []
    cur_index = 1
    for i in range(1,len(word_lens)-1):
      new_token = token_embeddings[cur_index:cur_index+word_lens[i],-4:,:]
      new_token = torch.sum(new_token, dim=1)
      new_token = torch.mean(new_token,0)
      
      new_token = new_token.flatten()
      list_token_embeddings.append(new_token)
      cur_index += word_lens[i]


    return list_token_embeddings

In [11]:
def prepare_data(train_inputs, train_labels, sent_len=20, embedding_size = 2304):
    train_labels = torch.tensor(train_labels)
    train_labels = train_labels.to(device)
    
    train_inputs = torch.concat((torch.zeros((sent_len//2-1,embedding_size),).to(device),
                                 torch.stack(train_inputs).to(device),
                                 torch.zeros((sent_len//2,embedding_size)).to(device)))
  
    train_labels = torch.concat((torch.zeros((sent_len//2-1),dtype=torch.long).to(device),
                                 train_labels,
                                 torch.zeros((sent_len//2),dtype=torch.long).to(device)))
    
    train_data = TensorDataset(train_inputs, train_labels)
    
    return train_data

In [47]:
def predict(model, val_data, sent_len, batch_size):
  model.eval()

  all_labels = []
  all_preds = []
  for i in range(0,len(val_data)-sent_len, batch_size):
      batch = [tuple(t.to(device) for t in val_data[i+b:i+sent_len+b]) for b in range(batch_size) if i+sent_len+b <= len(val_data)]

      b_input_ids = []
      b_labels = []

      for b in batch:
        input, labels = b
        if len(labels) != sent_len:
          print(labels)
        b_input_ids.append(input)
        all_labels.append(int(labels[sent_len//2-1].item()))
        b_labels.append(labels[sent_len//2-1])

      b_labels = torch.stack(b_labels, dim=0)
      b_input_ids = torch.stack(b_input_ids, dim=0)

      with torch.no_grad():
          logits = model(b_input_ids)

      preds = torch.argmax(logits, dim=1).flatten()

      all_preds+=[int(i.item()) for i in preds]

  return all_preds

In [46]:
def restoreText(words,punct):
  seps = [',','.','!','?']
  restored = words.copy()
  restored[0] = restored[0].title()
  p_index = np.argwhere(np.array(punct)!=0).flatten()
  for i in p_index:
    restored[i] = restored[i] + seps[punct[i]-1]
    if i < len(words)-1 and punct[i]-1!=0:
      restored[i+1] = restored[i+1].title()
  
  return ' '.join(restored)

In [50]:
# convert datasets to X and y
# X contains individual words. Numbers and punctuation marks are left out
# y contains classes for each word, a class is the punctuation mark that comes after a word. For example - "I like NLP!" is [0,0,2]

def process_text(text):
  X = []
  y = []

  tokens = word_tokenize(text)

  seps = [',','.','!','?']

  for token in tokens:

    if token in seps:
      sInd = seps.index(token)+1
      if sInd >= 2:
        y[-1] = 2
      else:
        y[-1] = 1
    else:
      X.append(token.lower())
      y.append(0)

  return X, y

##PREDICT

In [57]:
text = '''Он благополучно избегнул встречи с своею хозяйкой на лестнице. Каморка его приходилась под самою кровлей высокого пятиэтажного дома и походила более на шкаф, чем на квартиру. Квартирная же хозяйка его, у которой он нанимал эту каморку с обедом и прислугой, помещалась одною лестницей ниже, в отдельной квартире, и каждый раз, при выходе на улицу, ему непременно надо было проходить мимо хозяйкиной кухни, почти всегда настежь отворенной на лестницу. И каждый раз молодой человек, проходя мимо, чувствовал какое-то болезненное и трусливое ощущение, которого стыдился и от которого морщился. Он был должен кругом хозяйке и боялся с нею встретиться. Не то чтоб он был так труслив и забит, совсем даже напротив, но с некоторого времени он был в раздражительном и напряженном состоянии, похожем на ипохондрию.'''

In [58]:
X,y = process_text(text)

In [59]:
print(X)
print(y)

['он', 'благополучно', 'избегнул', 'встречи', 'с', 'своею', 'хозяйкой', 'на', 'лестнице', 'каморка', 'его', 'приходилась', 'под', 'самою', 'кровлей', 'высокого', 'пятиэтажного', 'дома', 'и', 'походила', 'более', 'на', 'шкаф', 'чем', 'на', 'квартиру', 'квартирная', 'же', 'хозяйка', 'его', 'у', 'которой', 'он', 'нанимал', 'эту', 'каморку', 'с', 'обедом', 'и', 'прислугой', 'помещалась', 'одною', 'лестницей', 'ниже', 'в', 'отдельной', 'квартире', 'и', 'каждый', 'раз', 'при', 'выходе', 'на', 'улицу', 'ему', 'непременно', 'надо', 'было', 'проходить', 'мимо', 'хозяйкиной', 'кухни', 'почти', 'всегда', 'настежь', 'отворенной', 'на', 'лестницу', 'и', 'каждый', 'раз', 'молодой', 'человек', 'проходя', 'мимо', 'чувствовал', 'какое-то', 'болезненное', 'и', 'трусливое', 'ощущение', 'которого', 'стыдился', 'и', 'от', 'которого', 'морщился', 'он', 'был', 'должен', 'кругом', 'хозяйке', 'и', 'боялся', 'с', 'нею', 'встретиться', 'не', 'то', 'чтоб', 'он', 'был', 'так', 'труслив', 'и', 'забит', 'совсем', 'д

In [60]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)
if use_cuda:
  model = modelPunct.cuda()
else:
  model = modelPunct.cpu()

cpu


In [61]:
%%time
tokenized_text, tokens_tensor, segments_tensors,word_lens = bert_text_preparation(" ".join(X), tokenizer)
embeddings = get_bert_embeddings(tokens_tensor.to(device), segments_tensors.to(device), word_lens, modelBert) 
data = prepare_data(embeddings,y,6,EMBEDDING_DIM)
preds = predict(modelPunct,data,6,len(y))

CPU times: user 1.02 s, sys: 606 µs, total: 1.02 s
Wall time: 1.05 s


In [62]:
print(text)
print(restoreText(X,preds))

Он благополучно избегнул встречи с своею хозяйкой на лестнице. Каморка его приходилась под самою кровлей высокого пятиэтажного дома и походила более на шкаф, чем на квартиру. Квартирная же хозяйка его, у которой он нанимал эту каморку с обедом и прислугой, помещалась одною лестницей ниже, в отдельной квартире, и каждый раз, при выходе на улицу, ему непременно надо было проходить мимо хозяйкиной кухни, почти всегда настежь отворенной на лестницу. И каждый раз молодой человек, проходя мимо, чувствовал какое-то болезненное и трусливое ощущение, которого стыдился и от которого морщился. Он был должен кругом хозяйке и боялся с нею встретиться. Не то чтоб он был так труслив и забит, совсем даже напротив, но с некоторого времени он был в раздражительном и напряженном состоянии, похожем на ипохондрию.
Он благополучно избегнул встречи с своею хозяйкой на лестнице. Каморка его приходилась под самою кровлей высокого пятиэтажного дома и походила более на шкаф, чем на квартиру. Квартирная же хозяйк